In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [2]:
from groq import Groq

In [3]:
def draft_message(content, role = 'user'):
  return {
      "role": role,
      "content": content,

  }
def truncate_content(content, max_length=4000):
    return content[:max_length]

In [5]:
client = Groq(api_key = api_key)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Dailyhunt_dataset/Telugu_Reference_Summary.csv')
df.head()
news_text = list(df['Content'])

In [8]:
from tqdm import tqdm
prompt = '''You are an AI assistant tasked with analyzing text and user comments to create concise summaries. Your summaries should integrate key information from all sources, maintaining the original language of the text. Focus on crucial points and minimize extraneous details.
            The summary should be in the same language as that of the original news. Present only the summary, no additional commentary or captions or titles.'''


In [9]:
from groq import Groq
import pandas as pd
# from google.colab import drive
from tqdm import tqdm

def draft_message(content, role='user'):
    return {
        "role": role,
        "content": content,
    }

def truncate_content(content, max_length=4500):
    return content[:max_length]

api_key = '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'
client = Groq(api_key=api_key)

# drive.mount('/content/drive')


df = pd.read_csv('/content/drive/MyDrive/Dailyhunt_dataset/Telugu_Reference_Summary.csv')

generated_headlines = []

for i in tqdm(range(len(news_text)), desc="Generating Headlines"):
    content = news_text[i]
    messages = [
        {
            "role": 'system',
            "content": f'''You are an AI assistant tasked with analyzing text and user comments to create concise summaries. Your summaries should integrate key information from all sources, maintaining the original language of the text. Focus on crucial points and minimize extraneous details.
            The summary should be in the same language as that of the original news. Present only the summary, no additional commentary or captions or titles.
            news: {content}'''
        }
    ]
    messages.append(draft_message(prompt))

    try:
        chat_completion = client.chat.completions.create(
            temperature=1.0,
            n=1,
            model='llama3-8b-8192',
            messages=messages
        )
        generated_headline = chat_completion.choices[0].message.content
    except Exception as e:
        error_message = str(e)
        if "context_length_exceeded" in error_message:
            print(f"Error generating headline for article {i+1}: Context length exceeded. Truncating content.")
            truncated_content = truncate_content(content)
            messages[0]['content'] = f'''news: {truncated_content}'''
            try:
                chat_completion = client.chat.completions.create(
                    temperature=1.0,
                    n=1,
                    model='llama3-8b-8192',
                    messages=messages
                )
                generated_headline = chat_completion.choices[0].message.content
            except Exception as e2:
                print(f"Error generating headline for article {i+1} after truncation: {e2}")
                generated_headline = "Error generating headline"
        else:
            print(f"Error generating headline for article {i+1}: {e}")
            generated_headline = "Error generating headline"

    generated_headlines.append(generated_headline)

df['Generated_summary'] = generated_headlines

df.to_csv('/content/drive/MyDrive/Dailyhunt_dataset/Telugu_Reference_Summary.csv', index=False)

print("done sal!")


Generating Headlines:   0%|          | 2/505 [00:44<3:13:59, 23.14s/it]

Error generating headline for article 2: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j240n558f1fr0d4gwgj41pp8` on tokens per minute (TPM): Limit 30000, Used 31230, Requested 1806. Please try again in 6.073s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Generating Headlines:   1%|          | 3/505 [00:56<2:38:09, 18.90s/it]


KeyboardInterrupt: 